In [6]:
import torch
import numpy as np
from PIL import Image
import os
import torch.nn.functional as F
from tqdm import tqdm
from scipy import interpolate
from pathlib import Path
import matplotlib.pyplot as plt
import cv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### E* Metric:

In [7]:
import gqvr.model.warp_utils as warp_utils
from gqvr.model.core_raft.utils.utils import InputPadder
from gqvr.model.core_raft.raft import RAFT

class RAFT_args:
    mixed_precision = True
    small = False
    alternate_corr = True # Reduces VRAM significantly in forward pass
    dropout = False

raft_args = RAFT_args()
raft_model = RAFT(raft_args)
raft_things_dict = torch.load("./pretrained_ckpts/models/raft-things.pth")
corrected_state_dict = {}
for k, v in raft_things_dict.items():
    k2 = ".".join(k.split(".")[1:])
    corrected_state_dict[k2] = v

raft_model.load_state_dict(corrected_state_dict)
raft_model.eval().requires_grad_(False).to(device)

def compute_video_warping_error(video_path):
    frames = os.listdir(video_path)
    frames.sort()
    frames = [np.array(Image.open(os.path.join(video_path, f)).convert('RGB').resize((512,512), Image.LANCZOS)) for f in frames]

    warping_error = 0
    err = 0
    
    Num = len(frames)
    tensor_frames = torch.stack([torch.from_numpy(frame) for frame in frames])
    # for i in range(Num):
    N = len(tensor_frames)
    indices = torch.linspace(0, N - 1, Num).long()
    extracted_frames = torch.index_select(tensor_frames, 0, indices)
    with torch.no_grad():
        for i in range(Num - 1):
            frame1 = extracted_frames[i]
            frame2 = extracted_frames[i + 1]

            # Calculate optical flow using Farneback method
            img1 = frame1.permute(2,0,1).float().unsqueeze(0).to(device)/ 255.0
            img2 = frame2.permute(2,0,1).float().unsqueeze(0).to(device)/ 255.0
            # img1 = torch.tensor(img2tensor(frame1)).float().to(device)
            # img2 = torch.tensor(img2tensor(frame2)).float().to(device)

            # Downsample the images by a factor of 2
            img1 = F.interpolate(img1, scale_factor=0.5, mode='bilinear', align_corners=False)
            img2 = F.interpolate(img2, scale_factor=0.5, mode='bilinear', align_corners=False)

            padder = InputPadder(img1.shape)
            img1, img2 = padder.pad(img1, img2)

            ### compute fw flow
            
            _, fw_flow = raft_model(img1, img2, iters=20, test_mode=True) # with optical flow model: RAFT
            fw_flow = warp_utils.tensor2img(fw_flow)
            # Clear cache and temporary data
            torch.cuda.empty_cache()

            ### compute bw flow
            _, bw_flow = raft_model(img2, img1, iters=20, test_mode=True) # with optical flow model: RAFT
            bw_flow = warp_utils.tensor2img(bw_flow)
            torch.cuda.empty_cache()

            ### compute occlusion
            fw_occ, warp_img2 = warp_utils.detect_occlusion(bw_flow, fw_flow, img2)
            warp_img2 = torch.tensor(warp_img2).float().to(device)
            fw_occ = torch.tensor(fw_occ).float().to(device)

            ### load flow
            flow = fw_flow

            ### load occlusion mask
            occ_mask = fw_occ
            noc_mask = 1 - occ_mask

            # ipdb.set_trace()   
            diff = (warp_img2- img1) * noc_mask
            diff_squared = diff ** 2

            
            # Calculate the sum and mean
            N = torch.sum(noc_mask)
            if N == 0:
                N = diff_squared.numel()
            # ipdb.set_trace()
            err += torch.sum(diff_squared) / N

    warping_error = err / (len(extracted_frames) - 1)

    return warping_error

## PSNR/SSIM from piq

In [8]:
import piq
def compute_full_reference_metrics(gt_img, out_img):
    # PSNR / SSIM
    psnr = piq.psnr(out_img, gt_img, data_range=1., reduction='none')
    ssim = piq.ssim(out_img, gt_img, data_range=1.) 
    # lpips = piq.LPIPS(reduction='none')(out_img, gt_img)
    return psnr.item(), ssim.item() # lpips.item()

## PATHs

In [9]:
SD21_burst_results = "/nobackup1/aryan/results/sd21_burst"
QUIVER_RESULTS = "/nobackup1/aryan/results/QUIVER"
QBP_RESULTS = "/nobackup1/aryan/results/QBP_BM3D"

scenes = ['xvfi_boat', 'xvfi_train', 
          'test_00014', 'test_00020', 'test_00021', 'test_00031',
          'small_jetengine', 'small_tank', 'small_explosion_0001', 'small_padlock', 'small_moreguns']

In [ ]:
cumulative_psnr = []
cumulative_ssim = []
cumulative_warping_error = []
pbar = tqdm(total=len(scenes))
for scene in scenes:
    test_dir_gt = SD21_burst_results + f"/{scene}_gt"
    gt_imgs = os.listdir(test_dir_gt)
    gt_imgs.sort()
    test_dir_out = SD21_burst_results + f"/{scene}_out"
    out_imgs = os.listdir(test_dir_out)
    out_imgs.sort()
    # test_dir_lq = SD21_burst_results + f"/{scene}_lq"
    psnr_list = []
    ssim_list = []
    warping_error = compute_video_warping_error(test_dir_out)
    for i in range(len(gt_imgs)):
        gt_img = Image.open(os.path.join(test_dir_gt, gt_imgs[i])).convert("RGB").resize((512,512), Image.LANCZOS)
        out_img = Image.open(os.path.join(test_dir_out, out_imgs[i])).convert("RGB").resize((512,512), Image.LANCZOS)
        gt_img = torch.tensor(np.array(gt_img)).permute(2,0,1).float().unsqueeze(0).to(device) / 255.0
        out_img = torch.tensor(np.array(out_img)).permute(2,0,1).float().unsqueeze(0).to(device) / 255.0
        psnr, ssim = compute_full_reference_metrics(gt_img, out_img)
        # print( psnr, ssim)
        psnr_list.append(psnr)
        ssim_list.append(ssim)
    pbar.set_description(f"Scene: {scene} | PSNR: {np.mean(psnr_list):.4f} | SSIM: {np.mean(ssim_list):.4f} | Warping Error: {warping_error:.4f}")
    print(f"{scene}: {np.mean(psnr_list):.4f}/{np.mean(ssim_list):.4f}/{warping_error:.4f}")
    cumulative_psnr.append(np.mean(psnr_list))
    cumulative_ssim.append(np.mean(ssim_list))
    cumulative_warping_error.append(warping_error.cpu().item())
    pbar.update(1)
pbar.close()

print(f"Average PSNR: {np.mean(cumulative_psnr):.4f}")
print(f"Average SSIM: {np.mean(cumulative_ssim):.4f}")
print(f"Average Warping Error: {np.mean(cumulative_warping_error):.4f}")

  0%|          | 0/11 [00:00<?, ?it/s]/tmp/ipykernel_1268855/3754814929.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  warp_img2 = torch.tensor(warp_img2).float().to(device)
Scene: xvfi_boat | PSNR: 27.5939 | SSIM: 0.7163 | Warping Error: 0.0017:   9%|▉         | 1/11 [00:06<01:09,  6.93s/it]

xvfi_boat: 27.5939/0.7163/0.0017


Scene: xvfi_train | PSNR: 24.0497 | SSIM: 0.7071 | Warping Error: 0.0034:  18%|█▊        | 2/11 [00:13<01:02,  6.96s/it]

xvfi_train: 24.0497/0.7071/0.0034


Scene: test_00014 | PSNR: 30.2209 | SSIM: 0.8299 | Warping Error: 0.0018:  27%|██▋       | 3/11 [00:57<03:10, 23.85s/it]

...

small_explosion_0001: 28.9755/0.8798/0.0044


Scene: small_padlock | PSNR: 34.6854 | SSIM: 0.9637 | Warping Error: 0.0014:  91%|█████████ | 10/11 [06:12<00:26, 26.54s/it]      

small_padlock: 34.6854/0.9637/0.0014


Scene: small_moreguns | PSNR: 31.7584 | SSIM: 0.9148 | Warping Error: 0.0011: 100%|██████████| 11/11 [06:53<00:00, 37.57s/it]

small_moreguns: 31.7584/0.9148/0.0011
Average PSNR: 29.8324
Average SSIM: 0.8556
Average Warping Error: 0.0021


In [ ]:
cumulative_psnr = []
cumulative_ssim = []
cumulative_warping_error = []
pbar = tqdm(total=len(scenes))
for scene in scenes:
    test_dir_gt = QUIVER_RESULTS + f"/{scene}_gt"
    gt_imgs = os.listdir(test_dir_gt)
    gt_imgs.sort()
    test_dir_out = QUIVER_RESULTS + f"/{scene}_out"
    out_imgs = os.listdir(test_dir_out)
    out_imgs.sort()
    # test_dir_lq = SD21_burst_results + f"/{scene}_lq"
    psnr_list = []
    ssim_list = []
    warping_error = compute_video_warping_error(test_dir_out)
    for i in range(len(gt_imgs)):
        gt_img = Image.open(os.path.join(test_dir_gt, gt_imgs[i])).convert("RGB").resize((512,512), Image.LANCZOS)
        out_img = Image.open(os.path.join(test_dir_out, out_imgs[i])).convert("RGB").resize((512,512), Image.LANCZOS)
        gt_img = torch.tensor(np.array(gt_img)).permute(2,0,1).float().unsqueeze(0).to(device) / 255.0
        out_img = torch.tensor(np.array(out_img)).permute(2,0,1).float().unsqueeze(0).to(device) / 255.0
        psnr, ssim = compute_full_reference_metrics(gt_img, out_img)
        # print( psnr, ssim)
        psnr_list.append(psnr)
        ssim_list.append(ssim)
    pbar.set_description(f"Scene: {scene} | PSNR: {np.mean(psnr_list):.4f} | SSIM: {np.mean(ssim_list):.4f} | Warping Error: {warping_error:.4f}")
    print(f"{scene}: {np.mean(psnr_list):.4f}/{np.mean(ssim_list):.4f}/{warping_error:.4f}")
    cumulative_psnr.append(np.mean(psnr_list))
    cumulative_ssim.append(np.mean(ssim_list))
    cumulative_warping_error.append(warping_error.cpu().item())
    pbar.update(1)
pbar.close()

print(f"Average PSNR: {np.mean(cumulative_psnr):.4f}")
print(f"Average SSIM: {np.mean(cumulative_ssim):.4f}")
print(f"Average Warping Error: {np.mean(cumulative_warping_error):.4f}")

  0%|          | 0/11 [01:24<?, ?it/s]
/tmp/ipykernel_1268855/3754814929.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  warp_img2 = torch.tensor(warp_img2).float().to(device)
Scene: xvfi_boat | PSNR: 20.8117 | SSIM: 0.6389 | Warping Error: 0.0009:   9%|▉         | 1/11 [00:07<01:13,  7.30s/it]

xvfi_boat: 20.8117/0.6389/0.0009


Scene: xvfi_train | PSNR: 25.1905 | SSIM: 0.8624 | Warping Error: 0.0018:  18%|█▊        | 2/11 [00:14<01:05,  7.30s/it]

xvfi_train: 25.1905/0.8624/0.0018


Scene: test_00014 | PSNR: 26.0922 | SSIM: 0.8566 | Warping Error: 0.0015:  27%|██▋       | 3/11 [01:00<03:20, 25.01s/it]

...

small_explosion_0001: 17.4564/0.5725/0.0021


Scene: small_padlock | PSNR: 24.8264 | SSIM: 0.9455 | Warping Error: 0.0016:  91%|█████████ | 10/11 [06:30<00:27, 27.80s/it]      

small_padlock: 24.8264/0.9455/0.0016


Scene: small_moreguns | PSNR: 15.2187 | SSIM: 0.7180 | Warping Error: 0.0019: 100%|██████████| 11/11 [07:13<00:00, 39.37s/it]

small_moreguns: 15.2187/0.7180/0.0019
Average PSNR: 22.4296
Average SSIM: 0.8137
Average Warping Error: 0.0017


In [ ]:
cumulative_psnr = []
cumulative_ssim = []
cumulative_warping_error = []
pbar = tqdm(total=len(scenes))
for scene in scenes:
    test_dir_gt = SD21_burst_results + f"/{scene}_gt"
    gt_imgs = os.listdir(test_dir_gt)
    gt_imgs.sort()
    test_dir_out = QBP_RESULTS + f"/{scene}"
    out_imgs = os.listdir(test_dir_out)
    out_imgs.sort()
    # test_dir_lq = SD21_burst_results + f"/{scene}_lq"
    psnr_list = []
    ssim_list = []
    warping_error = compute_video_warping_error(test_dir_out)
    for i in range(len(gt_imgs)):
        gt_img = Image.open(os.path.join(test_dir_gt, gt_imgs[i])).convert('L').resize((512,512), Image.LANCZOS)
        out_img = Image.open(os.path.join(test_dir_out, out_imgs[i])).convert('L').resize((512,512), Image.LANCZOS)
        gt_img = torch.tensor(np.array(gt_img)).float().unsqueeze(0).to(device) / 255.0
        out_img = torch.tensor(np.array(out_img)).float().unsqueeze(0).to(device) / 255.0
        psnr, ssim = compute_full_reference_metrics(gt_img.unsqueeze(0), out_img.unsqueeze(0))
        # print( psnr, ssim)
        psnr_list.append(psnr)
        ssim_list.append(ssim)
    pbar.set_description(f"Scene: {scene} | PSNR: {np.mean(psnr_list):.4f} | SSIM: {np.mean(ssim_list):.4f} | Warping Error: {warping_error:.4f}")
    print(f"{scene}: {np.mean(psnr_list):.4f}/{np.mean(ssim_list):.4f}/{warping_error:.4f}")
    cumulative_psnr.append(np.mean(psnr_list))
    cumulative_ssim.append(np.mean(ssim_list))
    cumulative_warping_error.append(warping_error.detach().cpu().item())
    pbar.update(1)
pbar.close()

print(f"Average PSNR: {np.mean(cumulative_psnr):.4f}")
print(f"Average SSIM: {np.mean(cumulative_ssim):.4f}")
print(f"Average Warping Error: {np.mean(cumulative_warping_error):.4f}")

Scene: xvfi_train | PSNR: 12.8230 | SSIM: 0.2287 | Warping Error: 0.0003:  18%|█▊        | 2/11 [00:45<03:24, 22.69s/it]
/tmp/ipykernel_1910057/3754814929.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  warp_img2 = torch.tensor(warp_img2).float().to(device)


xvfi_boat: 10.9781/0.4913/0.0007


xvfi_train: 13.0492/0.2487/0.0003


test_00014: 13.2860/0.5317/0.0006


test_00020: 18.6312/0.7070/0.0019


...

small_tank: 8.8698/0.4778/0.0023


small_explosion_0001: 19.7825/0.1607/0.0004


small_padlock: 11.2782/0.5458/0.0005


Scene: small_moreguns | PSNR: 9.2624 | SSIM: 0.6958 | Warping Error: 0.0024: 100%|██████████| 11/11 [12:52<00:00, 70.22s/it]

small_moreguns: 9.2624/0.6958/0.0024
Average PSNR: 13.3802
Average SSIM: 0.4477
Average Warping Error: 0.0010


In [5]:
SD21_Stage2_results = "/nobackup1/aryan/results/sd21/burst_testset_evaluation_s2"
cumulative_psnr = []
cumulative_ssim = []
cumulative_warping_error = []
pbar = tqdm(total=len(scenes))
for scene in scenes:
    test_dir_gt = SD21_burst_results + f"/{scene}_gt"
    gt_imgs = os.listdir(test_dir_gt)
    gt_imgs.sort()
    test_dir_out = SD21_Stage2_results + f"/{scene}"
    out_imgs = os.listdir(test_dir_out)
    out_imgs.sort()
    # test_dir_lq = SD21_burst_results + f"/{scene}_lq"
    psnr_list = []
    ssim_list = []
    warping_error = compute_video_warping_error(test_dir_out)
    for i in range(len(gt_imgs)):
        gt_img = Image.open(os.path.join(test_dir_gt, gt_imgs[i])).convert('L').resize((512,512), Image.LANCZOS)
        out_img = Image.open(os.path.join(test_dir_out, out_imgs[i])).convert('L').resize((512,512), Image.LANCZOS)
        gt_img = torch.tensor(np.array(gt_img)).float().unsqueeze(0).to(device) / 255.0
        out_img = torch.tensor(np.array(out_img)).float().unsqueeze(0).to(device) / 255.0
        psnr, ssim = compute_full_reference_metrics(gt_img.unsqueeze(0), out_img.unsqueeze(0))
        # print( psnr, ssim)
        psnr_list.append(psnr)
        ssim_list.append(ssim)
    pbar.set_description(f"Scene: {scene} | PSNR: {np.mean(psnr_list):.4f} | SSIM: {np.mean(ssim_list):.4f} | Warping Error: {warping_error:.4f}")
    print(f"{scene}: {np.mean(psnr_list):.4f}/{np.mean(ssim_list):.4f}/{warping_error:.4f}")
    cumulative_psnr.append(np.mean(psnr_list))
    cumulative_ssim.append(np.mean(ssim_list))
    cumulative_warping_error.append(warping_error.detach().cpu().item())
    pbar.update(1)
pbar.close()

print(f"Average PSNR: {np.mean(cumulative_psnr):.4f}")
print(f"Average SSIM: {np.mean(cumulative_ssim):.4f}")
print(f"Average Warping Error: {np.mean(cumulative_warping_error):.4f}")

  0%|          | 0/11 [00:00<?, ?it/s]/nobackup/aryan/.conda/envs/sd35/lib/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/tmp/ipykernel_1674845/3754814929.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  warp_img2 = torch.tensor(warp_img2).float().to(device)
Scene: xvfi_boat | PSNR: 27.4399 | SSIM: 0.7111 | Warping Error: 0.0018:   9%|▉         | 1/11 [00:07<01:15,  7.52s/it]

xvfi_boat: 27.4399/0.7111/0.0018


Scene: xvfi_train | PSNR: 24.2301 | SSIM: 0.7153 | Warping Error: 0.0036:  18%|█▊        | 2/11 [00:15<01:10,  7.82s/it]

xvfi_train: 24.2301/0.7153/0.0036


Scene: test_00014 | PSNR: 30.1029 | SSIM: 0.8293 | Warping Error: 0.0020:  27%|██▋       | 3/11 [01:06<03:38, 27.29s/it]

...

small_explosion_0001: 29.1489/0.8912/0.0049


Scene: small_padlock | PSNR: 34.8471 | SSIM: 0.9663 | Warping Error: 0.0014:  91%|█████████ | 10/11 [07:09<00:30, 30.92s/it]      

small_padlock: 34.8471/0.9663/0.0014


Scene: small_moreguns | PSNR: 31.5905 | SSIM: 0.9074 | Warping Error: 0.0011: 100%|██████████| 11/11 [07:57<00:00, 43.43s/it]

small_moreguns: 31.5905/0.9074/0.0011
Average PSNR: 29.8340
Average SSIM: 0.8570
Average Warping Error: 0.0023


In [ ]:
SD21_Stage1_results = "/nobackup1/aryan/results/sd21/burst_testset_evaluation_s1"
cumulative_psnr = []
cumulative_ssim = []
cumulative_warping_error = []
pbar = tqdm(total=len(scenes))
for scene in scenes:
    test_dir_gt = QUIVER_RESULTS + f"/{scene}_gt"
    gt_imgs = os.listdir(test_dir_gt)
    gt_imgs.sort()
    test_dir_out = SD21_Stage1_results + f"/{scene}"
    out_imgs = os.listdir(test_dir_out)
    out_imgs.sort()
    # test_dir_lq = SD21_burst_results + f"/{scene}_lq"
    psnr_list = []
    ssim_list = []
    warping_error = compute_video_warping_error(test_dir_out)
    for i in range(len(gt_imgs)):
        gt_img = Image.open(os.path.join(test_dir_gt, gt_imgs[i])).convert('L').resize((1024,1024), Image.LANCZOS)
        out_img = Image.open(os.path.join(test_dir_out, out_imgs[i])).convert('L').resize((1024,1024), Image.LANCZOS)
        gt_img = torch.tensor(np.array(gt_img)).float().unsqueeze(0).to(device) / 255.0
        out_img = torch.tensor(np.array(out_img)).float().unsqueeze(0).to(device) / 255.0
        psnr, ssim = compute_full_reference_metrics(gt_img.unsqueeze(0), out_img.unsqueeze(0))
        # print( psnr, ssim)
        psnr_list.append(psnr)
        ssim_list.append(ssim)
    pbar.set_description(f"Scene: {scene} | PSNR: {np.mean(psnr_list):.4f} | SSIM: {np.mean(ssim_list):.4f} | Warping Error: {warping_error:.4f}")
    print(f"{scene}: {np.mean(psnr_list):.4f}/{np.mean(ssim_list):.4f}/{warping_error:.4f}")
    cumulative_psnr.append(np.mean(psnr_list))
    cumulative_ssim.append(np.mean(ssim_list))
    cumulative_warping_error.append(warping_error.detach().cpu().item())
    pbar.update(1)
pbar.close()

print(f"Average PSNR: {np.mean(cumulative_psnr):.4f}")
print(f"Average SSIM: {np.mean(cumulative_ssim):.4f}")
print(f"Average Warping Error: {np.mean(cumulative_warping_error):.4f}")

Scene: xvfi_train | PSNR: 27.3034 | SSIM: 0.7572 | Warping Error: 0.0032:  18%|█▊        | 2/11 [00:32<02:26, 16.29s/it]
/tmp/ipykernel_1674845/3754814929.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  warp_img2 = torch.tensor(warp_img2).float().to(device)
Scene: xvfi_boat | PSNR: 16.9676 | SSIM: 0.7116 | Warping Error: 0.0013:   9%|▉         | 1/11 [00:07<01:11,  7.10s/it]

xvfi_boat: 16.9676/0.7116/0.0013


Scene: xvfi_train | PSNR: 24.0845 | SSIM: 0.7626 | Warping Error: 0.0032:  18%|█▊        | 2/11 [00:14<01:04,  7.13s/it]

xvfi_train: 24.0845/0.7626/0.0032


Scene: test_00014 | PSNR: 22.8988 | SSIM: 0.8112 | Warping Error: 0.0013:  27%|██▋       | 3/11 [00:58<03:14, 24.30s/it]

...

Scene: small_padlock | PSNR: 35.4416 | SSIM: 0.9707 | Warping Error: 0.0014:  91%|█████████ | 10/11 [06:19<00:27, 27.03s/it]      

small_padlock: 35.4416/0.9707/0.0014


Scene: small_moreguns | PSNR: 30.9350 | SSIM: 0.9499 | Warping Error: 0.0013: 100%|██████████| 11/11 [07:00<00:00, 38.26s/it]

small_moreguns: 30.9350/0.9499/0.0013
Average PSNR: 25.6290
Average SSIM: 0.8532
Average Warping Error: 0.0021


: 

In [19]:
EVAL_DIR = "/media/agarg54/Extreme SSD/code/gQVR/eval_s1_vs_s2_vs_s3"
scenes = ['test_00014', 'test_00021', 'test_00031', 'full_tank', 'small_moreguns']

In [21]:
def normalize(tens):
    return (tens - tens.min()) / (tens.max() - tens.min())

In [26]:
cumulative_psnr_s3 = []
cumulative_ssim_s3 = []
cumulative_warping_error_s3 = []

cumulative_psnr_s2 = []
cumulative_ssim_s2 = []
cumulative_warping_error_s2 = []

cumulative_psnr_s1 = []
cumulative_ssim_s1 = []
cumulative_warping_error_s1 = []

pbar = tqdm(total=len(scenes))
for scene in scenes:
    test_dir_gt = EVAL_DIR + f"/{scene}/gt"
    gt_imgs = os.listdir(test_dir_gt)
    gt_imgs.sort()
    test_dir_out_s3 = EVAL_DIR + f"/{scene}/out_s3"
    test_dir_out_s2 = EVAL_DIR + f"/{scene}/out_s2"
    test_dir_out_s1 = EVAL_DIR + f"/{scene}/out_s1"

    out_imgs_s3 = os.listdir(test_dir_out_s3)
    out_imgs_s3.sort()
    out_imgs_s2 = os.listdir(test_dir_out_s2)
    out_imgs_s2.sort()
    out_imgs_s1 = os.listdir(test_dir_out_s1)
    out_imgs_s1.sort()
    # test_dir_lq = SD21_burst_results + f"/{scene}_lq"
    psnr_list_s3 = []
    ssim_list_s3 = []
    psnr_list_s2 = []
    ssim_list_s2 = []
    psnr_list_s1 = []
    ssim_list_s1 = []

    if len(out_imgs_s3) > 1:
        warping_error_s3 = compute_video_warping_error(test_dir_out_s3)
        cumulative_warping_error_s3.append(warping_error_s3.detach().cpu().item())
        warping_error_s2 = compute_video_warping_error(test_dir_out_s2)
        cumulative_warping_error_s2.append(warping_error_s2.detach().cpu().item())
        warping_error_s1 = compute_video_warping_error(test_dir_out_s1)
        cumulative_warping_error_s1.append(warping_error_s1.detach().cpu().item())
    else:
        cumulative_warping_error_s1.append(0.)
        cumulative_warping_error_s2.append(0.)
        cumulative_warping_error_s3.append(0.)

    for i in range(len(gt_imgs)):
        gt_img = Image.open(os.path.join(test_dir_gt, gt_imgs[i])).convert('L').resize((512,512), Image.LANCZOS)
        out_img_s3 = Image.open(os.path.join(test_dir_out_s3, out_imgs_s3[i])).convert('L').resize((512,512), Image.LANCZOS)
        out_img_s2 = Image.open(os.path.join(test_dir_out_s2, out_imgs_s2[i])).convert('L').resize((512,512), Image.LANCZOS)
        out_img_s1 = Image.open(os.path.join(test_dir_out_s1, out_imgs_s1[i])).convert('L').resize((512,512), Image.LANCZOS)

        gt_img = torch.tensor(np.array(gt_img)).float().unsqueeze(0).to(device) / 255.0
        out_img_s3 = torch.tensor(np.array(out_img_s3)).float().unsqueeze(0).to(device) / 255.0
        out_img_s2 = torch.tensor(np.array(out_img_s2)).float().unsqueeze(0).to(device) / 255.0
        out_img_s1 = torch.tensor(np.array(out_img_s1)).float().unsqueeze(0).to(device) / 255.0

        gt_img, out_img_s2, out_img_s3, out_img_s1 = normalize(gt_img), normalize(out_img_s2), normalize(out_img_s3), normalize(out_img_s1)

        psnr_s3, ssim_s3 = compute_full_reference_metrics(gt_img.unsqueeze(0), out_img_s3.unsqueeze(0))
        psnr_s2, ssim_s2 = compute_full_reference_metrics(gt_img.unsqueeze(0), out_img_s2.unsqueeze(0))
        psnr_s1, ssim_s1 = compute_full_reference_metrics(gt_img.unsqueeze(0), out_img_s1.unsqueeze(0))
        # print( psnr, ssim)
        psnr_list_s3.append(psnr_s3)
        ssim_list_s3.append(ssim_s3)

        psnr_list_s2.append(psnr_s2)
        ssim_list_s2.append(ssim_s2)

        psnr_list_s1.append(psnr_s1)
        ssim_list_s1.append(ssim_s1)
    # pbar.set_description(f"Scene: {scene}")
    print(f"{scene} S3: {np.mean(psnr_list_s3):.4f}/{np.mean(ssim_list_s3):.4f}/{warping_error_s3:.4f}")
    print(f"{scene} S2: {np.mean(psnr_list_s2):.4f}/{np.mean(ssim_list_s2):.4f}/{warping_error_s2:.4f}")
    print(f"{scene} S1: {np.mean(psnr_list_s1):.4f}/{np.mean(ssim_list_s1):.4f}/{warping_error_s1:.4f}")
    cumulative_psnr_s3.append(np.mean(psnr_list_s3))
    cumulative_ssim_s3.append(np.mean(ssim_list_s3))
    cumulative_psnr_s2.append(np.mean(psnr_list_s2))
    cumulative_ssim_s2.append(np.mean(ssim_list_s2))
    cumulative_psnr_s1.append(np.mean(psnr_list_s1))
    cumulative_ssim_s1.append(np.mean(ssim_list_s1))
    pbar.update(1)
pbar.close()
print("-----------------------------------")
print("Ablation: S1 vs S2 vs S3 - On realistic sim bursts (PSNR/SSIM/E*)")
print("-----------------------------------")
print(f"Average PSNR S1: {np.mean(cumulative_psnr_s1):.4f}")
print(f"Average SSIM S1: {np.mean(cumulative_ssim_s1):.4f}")
print(f"Average Warping Error S1: {np.mean(cumulative_warping_error_s1)}")
print("-----------------------------------")
print(f"Average PSNR S2: {np.mean(cumulative_psnr_s2):.4f}")
print(f"Average SSIM S2: {np.mean(cumulative_ssim_s2):.4f}")
print(f"Average Warping Error S2: {np.mean(cumulative_warping_error_s2)}")
print("-----------------------------------")
print(f"Average PSNR S3: {np.mean(cumulative_psnr_s3):.4f}")
print(f"Average SSIM S3: {np.mean(cumulative_ssim_s3):.4f}")
print(f"Average Warping Error S3: {np.mean(cumulative_warping_error_s3)}")
# print(cumulative_warping_error_s2, cumulative_warping_error_s3)


  0%|          | 0/5 [00:00<?, ?it/s]/tmp/ipykernel_2160349/3754814929.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  warp_img2 = torch.tensor(warp_img2).float().to(device)
 20%|██        | 1/5 [00:03<00:15,  3.98s/it]

test_00014 S3: 26.9673/0.8088/0.0044
test_00014 S2: 24.5517/0.7345/0.0043
test_00014 S1: 24.4701/0.7606/0.0042


 40%|████      | 2/5 [00:07<00:11,  3.94s/it]

test_00021 S3: 24.9667/0.9157/0.0034
test_00021 S2: 27.0705/0.9225/0.0046
test_00021 S1: 22.1235/0.9088/0.0044


 60%|██████    | 3/5 [00:35<00:29, 14.57s/it]

test_00031 S3: 29.4883/0.8344/0.0059
test_00031 S2: 28.3928/0.8133/0.0070
test_00031 S1: 12.6355/0.3451/0.0101
full_tank S3: 26.0583/0.8620/0.0059
full_tank S2: 19.2660/0.8164/0.0070
full_tank S1: 19.4205/0.8245/0.0101


100%|██████████| 5/5 [00:37<00:00,  7.43s/it]

small_moreguns S3: 30.6675/0.9262/0.0263
small_moreguns S2: 21.2880/0.9436/0.0267
small_moreguns S1: 21.5380/0.9553/0.0268
-----------------------------------
Ablation: S1 vs S2 vs S3 - On realistic sim bursts (PSNR/SSIM/E*)
-----------------------------------
Average PSNR S1: 20.0375
Average SSIM S1: 0.7589
Average Warping Error S1: 0.009088441077619792
-----------------------------------
Average PSNR S2: 24.1138
Average SSIM S2: 0.8461
Average Warping Error S2: 0.008507703337818385
-----------------------------------
Average PSNR S3: 27.6296
Average SSIM S3: 0.8694
Average Warping Error S3: 0.008005498722195626
